Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Anumala Venu Madhava Reddy"
let rollno = "CS18B051"

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Monads

In this section, we will extend the state monad example from class to implement OCaml-like references. The idea is to use purely functional heaps using functions, and use this as the state used in the state monad. The definition of functional heap is below:

In [2]:
module type FHEAP = sig
  type ('k,'v) t
  val empty_heap : ('k,'v) t
  val set        : ('k,'v) t -> 'k -> 'v -> ('k,'v) t
  val get        : ('k,'v) t -> 'k -> 'v option
end

module FHeap : FHEAP = struct
  type ('k,'v) t = 'k -> 'v option
  let empty_heap = fun k -> None
  let set h x v = fun k -> if k = x then Some v else h k
  let get h x = h x
end

Here is how you can use the functional heap.

In [3]:
let open FHeap in 
let h = empty_heap in
let h = set h 0 "Hello" in
let h = set h 1 "World" in
Printf.printf "%s %s\n%!" (Option.get (get h 0)) (Option.get (get h 1));
let h = set h 1 "CS3100 students" in
Printf.printf "%s %s\n%!" (Option.get (get h 0)) (Option.get (get h 1))

Hello World
Hello CS3100 students


- : unit = ()


Observe that we have to thread through the heap `h` at every step. Just as we had done in the lecture, we will use a monad to implicitly thread through the heap through the comuputation. Here is the monad signature:

In [4]:
module type MONAD = sig
  type 'a t
  val return : 'a -> 'a t
  val (let*)  : 'a t -> ('a -> 'b t) -> 'b t
end

module type MONAD =
  sig
    type 'a t
    val return : 'a -> 'a t
    val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
  end


## Multiple References of the same type

First we will start with a monad that allows creation of new reference cells, read and update them but restricted to storing the same type of value. You will implement the `Ref_monad` module with the following signature:

In [5]:
module type REF_MONAD = sig
  type value   (* type of value *)
  type ref     (* type of reference *)
  include MONAD
  val mk_ref : value -> ref t
  val (!) : ref -> value t
  val (:=) : ref -> value -> unit t
  val run_state : 'a t -> 'a
end

module type REF_MONAD =
  sig
    type value
    type ref
    type 'a t
    val return : 'a -> 'a t
    val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
    val mk_ref : value -> ref t
    val ( ! ) : ref -> value t
    val ( := ) : ref -> value -> unit t
    val run_state : 'a t -> 'a
  end


Notice that `mk_ref` creates "fresh" reference cells. These reference cells are different from every other reference cell that has already been created or will be created in this monad. This property is known as **generativity**. We had seen a similar concept when we defined `fresh` for substitutions in the lambda calculus assignement (assignment 2). 

We can simulate freshness by using a counter value `c`, whose state is also maintained as part of the state. Everytime you want to create a fresh reference, update this counter `c`. We will use this counter to index into the functional heap. Hence, the reference itself (`ref` type) is just an integer. 

## Problem 1: Implement the Ref_monad

In [6]:
module Ref_monad (V : sig type t end)
  : REF_MONAD with type value = V.t = struct
  type value = V.t
  type ref = int (* just an index into the FHeap.t *)
  type 'a t = 
    int (* counter *) * (int, value) FHeap.t (* heap indexed by ref *) -> 
    int (* new counter *) * (int, value) FHeap.t (* new heap *) * 'a 
    
    

  let return a = fun (c,h) -> (c,h,a)
  
  
  let ( let* ) m f = fun (c,h) -> ( match m (c,h) with
                                  |(c',h',a') -> (f a') (c',h')
                                  ) 
                                  
  let mk_ref v = fun (c,h) -> (c+1,( FHeap.set h (c+1) v) , (c+1) )
  
  let (!) c = fun (c',h) -> (c',h,( Option.get (FHeap.get h c) ) )
  
  let (:=) c v = fun (c',h) -> (c',FHeap.set h c v , ())
  
  let run_state a = 
      begin match a (0,FHeap.empty_heap) with
      |(c,h,v) -> v
      end;
    
  
  
  (* Implement the missing functions *)
  (* YOUR CODE HERE *)
end

module Ref_monad :
  functor (V : sig type t end) ->
    sig
      type value = V.t
      type ref
      type 'a t
      val return : 'a -> 'a t
      val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
      val mk_ref : value -> ref t
      val ( ! ) : ref -> value t
      val ( := ) : ref -> value -> unit t
      val run_state : 'a t -> 'a
    end


In [7]:
(* 5 points *)
let module R = Ref_monad(struct type t = int end) in
let open R in
let module M = struct
  let program = 
    let* i = mk_ref 10 in
    !i
end
in assert (run_state M.program = 10)


- : unit = ()


In [8]:
(* 5 points *)
let module R = Ref_monad(struct type t = int end) in
let open R in 
let module M = struct
  let program = 
    let* i = mk_ref 10 in
    let* iv = !i in
    let* _ = (i := iv + 1) in
    !i
  end 
in assert (run_state M.program = 11)


- : unit = ()


In [9]:
(* 10 points *)
let module R = Ref_monad(struct type t = int end) in
let open R in
let module M = struct
  let program = 
    let* i = mk_ref 10 in
    let* j = mk_ref 20 in
    let* iv = !i in
    let* _ = i := iv + 1 in
    let* jv = !j in
    let* iv = !i in
    return (iv + jv)
end
in assert(run_state M.program = 31)


- : unit = ()


## Polymorphic Refences

Let us now graduate to implementing references of different types. It turns out that storing different typed values in a key-value map and retrieving it back is tricky; every `value` that is put into a given map must be the same. Luckily, we have a concept called universal types that provides a way to safely go from arbitrary types to the universal type and back. Here is an implementation of universal type in OCaml:

In [10]:
module Univ : sig
  type t
  type 'a packer = {pack : 'a -> t; unpack : t -> 'a option}
  val mk : unit -> 'a packer
end = struct
  type t = exn
  type 'a packer = {pack : 'a -> t; unpack : t -> 'a option}
  let mk : type a. unit -> a packer = fun () ->
    let module M () = struct exception E of a end in
    let module F = M () in
    {pack = (fun v -> F.E v); 
     unpack = fun p -> match p with F.E v -> Some v | _ -> None}
end

module Univ :
  sig
    type t
    type 'a packer = { pack : 'a -> t; unpack : t -> 'a option; }
    val mk : unit -> 'a packer
  end


The implementation details are not important. The `mk` function returns a packer for an arbitrary type, which includes an function to pack values of this arbitrary type to the universal type and unpack such a type. Here is how you might use the functions.

In [11]:
module M = struct
  include Univ
  let int_packer = mk ()
  let float_packer = mk ()
  
  let l = [int_packer.pack 10; float_packer.pack 20.25]
  
  let rec get packer l =
    match l with
    | [] -> None
    | x::xs -> 
        match packer.unpack x with
        | Some v -> Some v
        | None -> get packer xs
end;;

M.get M.int_packer M.l;;
M.get M.float_packer M.l


module M :
  sig
    type t = Univ.t
    type 'a packer =
      'a Univ.packer = {
      pack : 'a -> t;
      unpack : t -> 'a option;
    }
    val mk : unit -> 'a packer
    val int_packer : int packer
    val float_packer : float packer
    val l : t list
    val get : 'a packer -> t list -> 'a option
  end


- : int option = Some 10


- : float option = Some 20.25


The important thing to notice above is the type of `get` function. Given a `'a packer` and a list of universal values, the function retrieves the value packed with the given packer.

## Problem 2

Use the universal type to implement a polymorphic reference cell monad with the following signature. 

In [12]:
module type POLY_REF_MONAD = sig
  type 'a ref (* type of reference *)
  include MONAD
  val mk_ref : 'a -> 'a ref t
  val (!) : 'a ref -> 'a t
  val (:=) : 'a ref -> 'a -> unit t
  val run_state : 'a t -> 'a
end

module type POLY_REF_MONAD =
  sig
    type 'a ref
    type 'a t
    val return : 'a -> 'a t
    val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
    val mk_ref : 'a -> 'a ref t
    val ( ! ) : 'a ref -> 'a t
    val ( := ) : 'a ref -> 'a -> unit t
    val run_state : 'a t -> 'a
  end


Observe in the following that the reference cell is not just an integer but also includes the packer. Use this packer to pack the values before inserting into the heap and unpack after retrieving from the heap.

In [13]:
module Poly_ref_monad : POLY_REF_MONAD = struct
  type 'a ref = int * 'a Univ.packer
  type 'a t = int * (int, Univ.t) FHeap.t -> int * (int, Univ.t) FHeap.t * 'a
  
  open Univ
  
  let return a = fun (c,h) -> (c,h,a)
  
  let ( let*) m f = fun (c,h) -> ( match m (c,h) with
                                  |(c',h',a') -> (f a') (c',h')
                                  ) 
                                  
  let mk_ref v = let pac = mk()  in fun (c,h) -> (c+1, (FHeap.set h (c+1) (pac.pack v)  ) , (c+1,pac ) )
  
  let (!) c1 = 
      ( match c1 with
      |(c,pac) -> fun(c',h) -> (c',h,Option.get(pac.unpack(Option.get (FHeap.get h c ) ) ) )
      )
  
  let (:=) c2 v = 
      (match c2 with 
      |(c,pac) -> fun (c',h) -> (c', (FHeap.set h c (pac.pack v ))  ,() )
      )
  
  let run_state a = 
      begin match a (0,FHeap.empty_heap) with
      |(c,h,v) -> v
      end;
    
  

  (* Implement the missing functions *)

(* YOUR CODE HERE *)
end

module Poly_ref_monad : POLY_REF_MONAD


In [14]:
(* 10 points *)
let module M = struct
  open Poly_ref_monad

  let program = 
    let* i = mk_ref 10 in
    let* s = mk_ref "20" in
    return ()
  end 
in assert (Poly_ref_monad.run_state M.program = ())


- : unit = ()


In [15]:
(* 10 points *)
let open Poly_ref_monad in
let module M = struct
  let program = 
    let* i = mk_ref 10 in
    let* s = mk_ref "20" in
    let* iv = !i in
    let* _ = i := iv + 1 in
    let* sv = !s in
    let* iv = !i in
    return (iv + int_of_string sv)
end
in assert (run_state M.program = 31)


- : unit = ()


# Streams

Consider the standard lazy stream definition.

In [16]:
type 'a stream = Cons of 'a * 'a stream Lazy.t

let hd (Cons(x,_)) = x
let tl (Cons(x,xs)) = Lazy.force xs
let rec take n s =
  if n = 0 then []
  else hd s::(take (n-1) (tl s))

type 'a stream = Cons of 'a * 'a stream Lazy.t


val hd : 'a stream -> 'a = <fun>


val tl : 'a stream -> 'a stream = <fun>


val take : int -> 'a stream -> 'a list = <fun>


## Problem 3

Define a value `pow2 : int stream` whose elements are the powers of two: `<1; 2; 4; 8; 16, ...>`

In [17]:
let mult a b = a*b

let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), lazy(zip f (tl s1) (tl s2)) )

let rec twos = Cons(2, lazy(twos))

let rec pow2 = Cons(1,lazy(Cons(2,lazy( zip mult twos (tl pow2) ))))

val mult : int -> int -> int = <fun>


val zip : ('a -> 'b -> 'c) -> 'a stream -> 'b stream -> 'c stream = <fun>


val twos : int stream = Cons (2, <cycle>)


val pow2 : int stream = Cons (1, <lazy>)


In [18]:
(* 10 points *)
assert (List.hd (List.rev (take 10 pow2)) = 512)

- : unit = ()


## Problem 4

Define a function `interleave : 'a stream -> 'a stream -> 'a stream`, such that `interleave <a1; a2; a3; ...> <b1; b2; b3; ...>` is the stream `<a1; b1; a2; b2; a3; b3; ...>`. For example, `interleave nats pow2` would be `<0; 1; 1; 2; 2; 4; 3; 8; ...>`.

In [19]:
let inter x y count = if count mod 2 = 0 then x else y

let rec join s1 s2 count = Cons(inter (hd s1) (hd s2) count,lazy(join (tl s1) (tl s2 ) (count+1) ))

let interleave s1 s2 = join s1 s2 0


val inter : 'a -> 'a -> int -> 'a = <fun>


val join : 'a stream -> 'a stream -> int -> 'a stream = <fun>


val interleave : 'a stream -> 'a stream -> 'a stream = <fun>


In [20]:
(* 15 points *)
let rec zeros = Cons(0,lazy zeros) in
let rec ones = Cons(1,lazy ones) in
assert (take 10 (interleave zeros ones) = [0; 1; 0; 1; 0; 1; 0; 1; 0; 1])

- : unit = ()


## Approximately e

The exponential function  $e^x$  can be computed by the following infinite sum:

\\[
e^x = \frac{x^0}{0!} + \frac{x^1}{1!} + \frac{x^2}{2!} + \ldots + \frac{x^k}{k!} + \ldots
\\]

## Problem 5

Define a function `e_terms : float -> float stream`. Element `k` of the stream should be term `k` from the infinite sum. For example, `e_terms 1.0` is the stream `<1.0; 1.0; 0.5; 0.1666...; 0.041666...; ...>`. The easy way to compute that involves a function that computes $f(k)=\frac{x^k}{k!}$. 

In [21]:
let div a b = Float.div a b

let mult a b = Float.mul a b

let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), lazy(zip f (tl s1) (tl s2)) )

   (* let rec twos = Cons(2, lazy(twos))  *)

let  pow x  = let rec  pox = Cons(x,lazy(pox)) in pox

let nume x = let rec powof = Cons(1.,lazy(Cons(x,lazy( zip mult (pow x) (tl powof) )))) in powof

let rec from n = Cons (n, lazy( from (n+.1.)) )

let rec denom = Cons(1.,lazy(  Cons(2.,lazy(zip mult (from 3.) (tl denom)))  ))

let denum = Cons(1.,lazy(denom))

let e_terms x = zip div (nume x) (denum)

val div : float -> float -> float = <fun>


val mult : float -> float -> float = <fun>


val zip : ('a -> 'b -> 'c) -> 'a stream -> 'b stream -> 'c stream = <fun>


val pow : 'a -> 'a stream = <fun>


val nume : float -> float stream = <fun>


val from : float -> float stream = <fun>


val denom : float stream = Cons (1., <lazy>)


val denum : float stream = Cons (1., lazy (Cons (1., <lazy>)))


val e_terms : float -> float stream = <fun>


In [22]:
(* 10 points *)
assert (Float.round(List.hd (List.rev (take 10 (e_terms 1.0))) *. (10. ** 9.)) = 2756.)

- : unit = ()


## Problem 6

Define a function `total : float stream -> float stream`, such that `total <a; b; c; ...>` is a running total of the input elements, i.e., `<a; a+.b; a+.b+.c; ...>`.

In [23]:
let add a b = Float.add a b

let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), lazy(zip f (tl s1) (tl s2)) )

let total s = let rec tem = Cons((hd s),lazy(Cons((hd s) +. (hd (tl s)),
                                            lazy(zip add (tl(tl s)) (tl(tem))  ) )) ) in tem

val add : float -> float -> float = <fun>


val zip : ('a -> 'b -> 'c) -> 'a stream -> 'b stream -> 'c stream = <fun>


val total : float stream -> float stream = <fun>


In [24]:
(* 15 points *)
let rec f x = Cons (x, lazy (f (x+.1.0))) in
assert (take 10 (total (f 1.0)) = [1.; 3.; 6.; 10.; 15.; 21.; 28.; 36.; 45.; 55.])

- : unit = ()


## Problem 7

Define a function `within : float -> float stream -> float`, such that `within eps s` is the first element of `s` for which the absolute difference between that element and the element before it is strictly less than `eps`. If there is no such element, within is permitted not to terminate (i.e., go into an "infinite loop"). As a precondition, the tolerance `eps` must be strictly positive. For example, `within 0.1 <1.0; 2.0; 2.5; 2.75; 2.875; 2.9375; 2.96875; ...>` is `2.9375`.

In [25]:
let sub a b = Float.add a (Float.neg b)

let newstream s = zip sub (tl s) s

let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), lazy(zip f (tl s1) (tl s2)) )

let rec check eps s s' = if((hd s') < eps) then hd(s) else check eps (tl s) (tl s')
                        
let within eps s = check eps s (newstream s)

val sub : float -> float -> float = <fun>


val newstream : float stream -> float stream = <fun>


val zip : ('a -> 'b -> 'c) -> 'a stream -> 'b stream -> 'c stream = <fun>


val check : 'a -> 'b stream -> 'a stream -> 'b = <fun>


val within : float -> float stream -> float = <fun>


In [26]:
(* 20 points *)
assert (Float.round(within 1e-15 (total (e_terms 1.0)) *. (10. ** 3.)) = 2718.)

- : unit = ()


## Problem 8

Finally, define a function `e : float -> float -> float` such that `e x eps` is  $e^x$  computed to within a tolerance of `eps`, which must be strictly positive. Note that there is an interesting boundary case where `x=1.0` for the first two terms of the sum; you could choose to drop the first term (which is always 1.0) from the stream before using `within`.

In [27]:
let e x eps =  within eps (total (e_terms x))

val e : float -> float -> float = <fun>


In [28]:
(* 10 points *)
assert (Float.round (e 1.0 1e-15 *. (10. ** 6.)) = 2718282.)

- : unit = ()
